<a href="https://colab.research.google.com/github/aecoaker/FTA-Summary/blob/master/Creating%20Final%20Summaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Summarisation

# Getting model set up

In [4]:
#import the model
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
# load a pre-trained model and tokenizer 'bart-large-cnn'
tokeniser = BartTokenizer.from_pretrained('facebook/bart-base')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [61]:
#import other libraries
import pandas as pd
import pickle

In [ ]:
#write in function developed previously for splitting into sections (or chunks) that are not too long to be fed into BART
def summary_chunks(text):
    chunks = []
    #split and join back together to remove odd whitespaces
    remaining_text = " ".join(text.split())
    #loop through and split text off into the list of chunks
    while remaining_text.count('.') > 1:
      words = remaining_text.split()
      stop_index = " ".join(words[0:750]).rfind('.')
      chunks.append(remaining_text[0:stop_index+1])
      remaining_text = remaining_text[stop_index+1:]
    #have to remove some specific sections which tokenise into too many words in an unhelpful way in BART's tokeniser
    chunks = [c.replace('𝑅𝑅𝑅𝑅𝑅𝑅 = 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑡𝑡ℎ𝑣𝑣 𝑔𝑔𝑜𝑜𝑜𝑜𝑔𝑔 − 𝑣𝑣𝑣𝑣 𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑛𝑛𝑜𝑜𝑛𝑛−𝑜𝑜𝑜𝑜𝑜𝑜𝑔𝑔𝑜𝑜𝑛𝑛𝑣𝑣𝑡𝑡𝑜𝑜𝑛𝑛𝑔𝑔 𝑚𝑚𝑣𝑣𝑡𝑡𝑣𝑣𝑜𝑜 𝑜𝑜𝑣𝑣𝑣𝑣𝑚𝑚 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑡𝑡ℎ𝑣𝑣 𝑔𝑔𝑜𝑜𝑜𝑜𝑔𝑔', '') for c in chunks]
    chunks = [c.replace('𝑅𝑅𝑅𝑅𝑅𝑅 = 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑜𝑜𝑜𝑜𝑜𝑜𝑔𝑔𝑜𝑜𝑛𝑛𝑣𝑣𝑡𝑡𝑜𝑜𝑛𝑛𝑔𝑔 𝑚𝑚𝑣𝑣𝑡𝑡𝑣𝑣𝑜𝑜 𝑜𝑜𝑣𝑣𝑣𝑣𝑚𝑚 𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣𝑣 𝑜𝑜𝑜𝑜 𝑡𝑡ℎ𝑣𝑣 𝑔𝑔𝑜𝑜𝑜𝑜𝑔𝑔×100', '') for c in chunks]
    return chunks

In [9]:
articles_df = pd.read_csv("article_topics.csv")

In [41]:
#generate summaries for app
l = 0.9
summaries = []
pre_recursive_summaries = []
for t in range(20):
  example_classification = articles_df[articles_df[('prob_t' + str(t))] >= l]
  example_topic_text = ' '.join(example_classification['Article Text'])
  example_text_chunks = summary_chunks(example_topic_text)
  inputs = [tokeniser.batch_encode_plus([text],return_tensors = 'pt') for text in example_text_chunks]
  summary_ids = [model.generate(i['input_ids'], early_stopping = True) for i in inputs]
  bart_summaries = [tokeniser.decode(ids[0], skip_special_tokens = True) for ids in summary_ids]
  single_sum = ' '.join(bart_summaries)
  pre_recursive_summaries.append(single_sum)
  input = tokeniser.batch_encode_plus([single_sum],return_tensors='pt')
  summary_id = model.generate(input['input_ids'], early_stopping=True)
  final_sum = tokeniser.decode(summary_id[0], skip_special_tokens=True)
  print(str(t)) #print the step in order to keep track of progress
  summaries.append(final_sum)
print(summaries)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
['Except as provided in this Section, e ach Party affirms its rights and obligations', 'Neither Party shall adopt or maintain, either on the basis of its entire territory or on', 'Each Party shall accord national treatment to the goods of the other Party in accordance with Article', 'A Party shall not reject a declaration of origin due to minor errors or discrepancies, such', 'Expressions The Parties affirm their commitment to work together through discussion and by the exchange of', '1. If, as a result of customs duty reduction or elimination, an originating good', '1. The Parties recognise that a broad range of mechanisms exists to facilitate the acceptance in', 'Neither Party shall have recourse to dispute settlement under Chapter 30 (Dispute Settlement)', '1. This Chapter applies to measures of a Party affecting the supply of professional services,', 'For the purposes of this Chapter, the value of a material is: (a)', 'Each Party sh

In [42]:
#review the outputs
summaries

['Except as provided in this Section, e ach Party affirms its rights and obligations',
 'Neither Party shall adopt or maintain, either on the basis of its entire territory or on',
 'Each Party shall accord national treatment to the goods of the other Party in accordance with Article',
 'A Party shall not reject a declaration of origin due to minor errors or discrepancies, such',
 'Expressions The Parties affirm their commitment to work together through discussion and by the exchange of',
 '1. If, as a result of customs duty reduction or elimination, an originating good',
 '1. The Parties recognise that a broad range of mechanisms exists to facilitate the acceptance in',
 'Neither Party shall have recourse to dispute settlement under Chapter 30 (Dispute Settlement)',
 '1. This Chapter applies to measures of a Party affecting the supply of professional services,',
 'For the purposes of this Chapter, the value of a material is: (a)',
 'Each Party shall ensure that, when its state- owned e

The summaries are being cut off by the default value of some length parameter, we need to fix this to create reasonable results.

In [43]:
#review the pre-resursive summaries to see if they are having the same issue
pre_recursive_summaries

['Except as provided in this Section, e ach Party affirms its rights and obligations  Each Party shall provide a system for the examination and registration of trade marks which includes among',
 "Neither Party shall adopt or maintain, either on the basis of its entire territory or on 8 For the purposes of this Chapter: “cross -border financial service supplier�  Financial services include the following activities: Insurance and insurance- related services (a) direct . 2 For greater certainty, the Parties understand that an investor “attempts  A Party shall not adopt or maintain, with respect to: (a) an established . Paragraph 1 does not permit suppliers of the services listed in subparagraphs 1( 1 (Definitions) with respect to: (a) insurance of risks relating to  The Parties recognise that promoting cooperation between the Parties and their financial regulatory authorities supports several objectives . (b) In the areas where one of the Parties defers to the regulatory  The framework f

In [54]:
#issue clearly effecting both sets of summaries
#therefore edit max-length parameters at both steps
l = 0.9
summaries_1 = []
pre_recursive_summaries_1 = []
for t in range(20):
  example_classification = articles_df[articles_df[('prob_t' + str(t))] >= l]
  example_topic_text = ' '.join(example_classification['Article Text'])
  example_text_chunks = summary_chunks(example_topic_text)
  inputs = [tokeniser.batch_encode_plus([text],return_tensors = 'pt') for text in example_text_chunks]
  summary_ids = [model.generate(i['input_ids'], min_new_tokens = 30, max_new_tokens = 50, early_stopping = True) for i in inputs]
  bart_summaries = [tokeniser.decode(ids[0], skip_special_tokens = True) for ids in summary_ids]
  single_sum = ' '.join(bart_summaries)
  pre_recursive_summaries_1.append(single_sum)
  input = tokeniser.batch_encode_plus([single_sum],return_tensors='pt')
  summary_id = model.generate(input['input_ids'], min_new_tokens = 10, max_new_tokens = 120, early_stopping=True)
  final_sum = tokeniser.decode(summary_id[0], skip_special_tokens=True)
  print(str(t)) #print the step in order to keep track of progress
  summaries_1.append(final_sum)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
['Except as provided in this Section, e ach Party affirms its rights and obligations', 'Neither Party shall adopt or maintain, either on the basis of its entire territory or on', 'Each Party shall accord national treatment to the goods of the other Party in accordance with Article', 'A Party shall not reject a declaration of origin due to minor errors or discrepancies, such', 'Expressions The Parties affirm their commitment to work together through discussion and by the exchange of', '1. If, as a result of customs duty reduction or elimination, an originating good', '1. The Parties recognise that a broad range of mechanisms exists to facilitate the acceptance in', 'Neither Party shall have recourse to dispute settlement under Chapter 30 (Dispute Settlement)', '1. This Chapter applies to measures of a Party affecting the supply of professional services,', 'For the purposes of this Chapter, the value of a material is: (a)', 'Each Party sh

In [55]:
summaries_1

['Except as provided in this Section, e ach Party affirms its rights and obligations under Article VI of GATT 1994, the AD Agreement, and the SCM Agreement. 1. Each Party shall issue, prior to the importation of a trade mark, a list of trade marks for examination and registration of trade mark. 2. Each party shall provide a system for the examination and register of trade Marks which includes among other things: (a) communicating to the applicant in writing, which may be by electronic means, the reasons for any refusal to register a trade Mark;',
 'Neither Party shall adopt or maintain, either on the basis of its entire territory or on the grounds of the territory of a central, regional or local level of government, a measure that: (a) imposes a limitation on: (i) 8 For the purposes of this Chapter: “cross -border financial service supplier” means a person of a Party that is engaged in the business of supplying a financial service within the territoryof the Party and that seeks to supp

In [58]:
#issue clearly effecting both sets of summaries
#therefore edit max-length parameters at both steps
l = 0.9
summaries_2 = []
pre_recursive_summaries_2 = []
for t in range(20):
  example_classification = articles_df[articles_df[('prob_t' + str(t))] >= l]
  example_topic_text = ' '.join(example_classification['Article Text'])
  example_text_chunks = summary_chunks(example_topic_text)
  inputs = [tokeniser.batch_encode_plus([text],return_tensors = 'pt') for text in example_text_chunks]
  summary_ids = [model.generate(i['input_ids'], min_new_tokens = 30, max_new_tokens = 50, early_stopping = True) for i in inputs]
  bart_summaries = [tokeniser.decode(ids[0], skip_special_tokens = True) for ids in summary_ids]
  single_sum = ' '.join(bart_summaries)
  pre_recursive_summaries_2.append(single_sum)
  input = tokeniser.batch_encode_plus([single_sum],return_tensors='pt')
  summary_id = model.generate(input['input_ids'], min_new_tokens = 10, max_new_tokens = 200, early_stopping=True)
  final_sum = tokeniser.decode(summary_id[0], skip_special_tokens=True)
  print(str(t)) #print the step in order to keep track of progress
  summaries_2.append(final_sum)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [59]:
summaries_2

['Except as provided in this Section, e ach Party affirms its rights and obligations under Article VI of GATT 1994, the AD Agreement, and the SCM Agreement. 1. Each Party shall issue, prior to the importation of a trade mark, a list of trade marks for examination and registration of trade mark. 2. Each party shall provide a system for the examination and register of trade Marks which includes among other things: (a) communicating to the applicant in writing, which may be by electronic means, the reasons for any refusal to register a trade Mark;',
 'Neither Party shall adopt or maintain, either on the basis of its entire territory or on the grounds of the territory of a central, regional or local level of government, a measure that: (a) imposes a limitation on: (i) 8 For the purposes of this Chapter: “cross -border financial service supplier” means a person of a Party that is engaged in the business of supplying a financial service within the territoryof the Party and that seeks to supp

This isn't perfect but not because of any limit on max lengths. The text output appears not to be fluent enough to finish all sentences in a gramatically correct way.

# Generating summaries

In [63]:
#output results to be used
with open('summaries.pkl', 'wb') as f:
    pickle.dump(summaries_2, f)
with open('pre_recursive_summaries.pkl', 'wb') as f:
    pickle.dump(pre_recursive_summaries_2, f)